In [1]:
!pip3 install mysql-connector-python

You should consider upgrading via the 'c:\anaconda\python.exe -m pip install --upgrade pip' command.


In [1]:
import mysql.connector
import pandas as pd
import random

Прочитаем данные с существующими студентами, преподавателями и предметами

In [60]:
students = pd.read_csv('students.csv', sep=';')
teachers = pd.read_csv('teachers.csv', sep=';')
subjects = pd.read_csv('subjects.csv', sep=';')

In [61]:
students.head(2)

,Фамилия,Имя,Гендер,Возраст
0,Ацтеков,Тотем,М,12
1,Васькин,Дима,М,17


In [62]:
teachers.head(2)

,Фамилия,Имя,Отчество,Гендер
0,Мемовский,Игнатий,Игоревич,М
1,Доро,Гупеда-Гогу,-,Ж


In [63]:
subjects.head(2)

,Название,Преподаватели
0,Телеграмоведение,"Мемовский Игнатий Игоревич, Алио Елена Ивановна"
1,Твитоинжениринг,"Доро Гупеда-Гогу, Алио Елена Ивановна"


![title](architecture.png)

Подключаемся к БД

In [2]:
con = mysql.connector.connect(host='127.0.0.1', port=33062, user='root', password='my-secret-pw')

In [3]:
cur=con.cursor(dictionary=True)

In [5]:
# cur.execute('''CREATE DATABASE school_database /*!40100 DEFAULT CHARACTER 
#             SET utf8mb4 COLLATE utf8mb4_0900_ai_ci */ /*!80016 DEFAULT ENCRYPTION='N' */;''')

In [6]:
con = mysql.connector.connect(host='127.0.0.1', port=33062, database='school_database', user='root', password='my-secret-pw')

In [7]:
cur=con.cursor(dictionary=True)

In [119]:
# cur.execute("""
# DROP TABLE IF EXISTS teachers2subjects
# """)
# cur.execute("""
# DROP TABLE IF EXISTS grades
# """)
# cur.execute("""
# DROP TABLE IF EXISTS students
# """)
# cur.execute("""
# DROP TABLE IF EXISTS teachers
# """)
# cur.execute("""
# DROP TABLE IF EXISTS subjects
# """)

In [104]:
cur.execute("""CREATE TABLE students (
              id_student INT NOT NULL AUTO_INCREMENT,
              last_name VARCHAR(256) NOT NULL,
              first_name VARCHAR(256) NOT NULL,
              gender VARCHAR(1) NULL,
              age INT NOT NULL,
              PRIMARY KEY (id_student),
              UNIQUE INDEX id_student_UNIQUE (id_student ASC) VISIBLE);
""")

cur.execute("""CREATE TABLE teachers (
              id_teacher INT NOT NULL AUTO_INCREMENT,
              last_name VARCHAR(256) NOT NULL,
              first_name VARCHAR(256) NOT NULL,
              patronymic VARCHAR(256) NOT NULL,
              gender VARCHAR(1) NULL,
              PRIMARY KEY (id_teacher),
              UNIQUE INDEX id_teacher_UNIQUE (id_teacher ASC) VISIBLE);
""")

cur.execute("""CREATE TABLE subjects (
              id_subject INT NOT NULL AUTO_INCREMENT,
              name VARCHAR(512) NOT NULL,
              PRIMARY KEY (id_subject),
              UNIQUE INDEX id_subject_UNIQUE (id_subject ASC) VISIBLE);
""")

Перед тем, как заполнять таблицы данными, создадим ещще две - для сопоставления предмета и преподавателя (many2many) и для записывания оценок

In [105]:
cur.execute("""CREATE TABLE teachers2subjects (
              id_teach2subj INT PRIMARY KEY NOT NULL AUTO_INCREMENT,
              id_teacher INT NOT NULL,
              id_subject INT NOT NULL,
              UNIQUE INDEX id_teach2subj_UNIQUE (id_teach2subj ASC) VISIBLE,
              FOREIGN KEY(id_teacher) REFERENCES teachers(id_teacher),
              FOREIGN KEY(id_subject) REFERENCES subjects(id_subject));
""")

cur.execute("""CREATE TABLE grades (
              id_grade INT PRIMARY KEY NOT NULL AUTO_INCREMENT,
              id_student INT NOT NULL,
              id_subject INT NOT NULL,
              grade INT NOT NULL,
              UNIQUE INDEX id_grade_UNIQUE (id_grade ASC) VISIBLE,
              FOREIGN KEY(id_student) REFERENCES students(id_student),
              FOREIGN KEY(id_subject) REFERENCES subjects(id_subject));
""")

Читаем данные из списка студентов и преподавателей и заполняем таблицу

In [106]:
for idx, student in students.iterrows():

    sss = f'''INSERT INTO students (last_name, first_name, gender, age) 
              VALUES ('{student['Фамилия']}', '{student['Имя']}', '{student['Гендер']}', '{student['Возраст']}')'''
    
    cur.execute(sss)
    
    if idx % 5 == 0:
        con.commit()

In [107]:
for idx, teacher in teachers.iterrows():

    sss = f'''INSERT INTO teachers (last_name, first_name, patronymic, gender) 
              VALUES ('{teacher['Фамилия']}', '{teacher['Имя']}', '{teacher['Отчество']}', '{teacher['Гендер']}')'''
    
    cur.execute(sss)
    
    if idx % 5 == 0:
        con.commit()

Читаем список предметов и сразу заполняем таблицу со списком предметов и teacher2subject

In [108]:
def get_teachers_ids(teachers: str):
    subj_teachers = teachers.split(', ')
    
    ids = []
    for teach in subj_teachers:
        names = teach.split()
        if len(names) < 3:
            names.append('-')
        
        cur.execute(f'''SELECT id_teacher FROM (teachers)
                       WHERE (last_name = '{names[0]}')
                       AND (first_name = '{names[1]}')
                       AND (patronymic = '{names[2]}')''')
        subj_id = cur.fetchall()[0]['id_teacher']
        ids.append(subj_id)
    
    return ids

In [109]:
for idx, subject in subjects.iterrows():

    sss = f'''INSERT INTO subjects (name) VALUES ('{subject['Название']}')'''
    
    cur.execute(sss)
    con.commit()
    
    cur.execute('''SELECT MAX(id_subject) FROM subjects''')
    subj_id = cur.fetchall()[0]['MAX(id_subject)']
    
    teachers_ids = get_teachers_ids(subject['Преподаватели'])
    
    for teach_id in teachers_ids:
        sss = f'''INSERT INTO teachers2subjects (id_teacher, id_subject) VALUES ('{teach_id}', '{subj_id}')'''
    
        cur.execute(sss)
    
    con.commit()

Теперь "проставим" всем ученикам оценки по всем предметам (будем считать, что это таблица итоговых оценок, и у каждого студента за предмет только одна оценка)

In [110]:
cur.execute('''SELECT id_student FROM students''')
all_student_ids = [student['id_student'] for student in cur.fetchall()]

cur.execute('''SELECT id_subject FROM subjects''')
all_subj_ids = [subject['id_subject'] for subject in cur.fetchall()]

In [111]:
for subj_id in all_subj_ids:
    for stud_id in all_student_ids:
        sss = f'''INSERT INTO grades (id_student, id_subject, grade) VALUES ('{stud_id}', '{subj_id}', '{random.randint(1, 10)}')'''
        cur.execute(sss)
    
    con.commit()

## Запросы

Выведем названия предметов по которым средняя оценка выше 5

In [8]:
query = '''
SELECT * 
FROM
    (
    SELECT subjects.name, avg(grades.grade) as avg_grade 
    FROM school_database.grades 
    JOIN school_database.subjects 
    ON grades.id_subject = subjects.id_subject
    GROUP BY grades.id_subject
    ) avg_subj_grades
WHERE avg_grade > 5;
'''

In [9]:
cur.execute(query)
subj_over_5 = cur.fetchall()

In [10]:
subj_over_5

[{'name': 'Телеграмоведение', 'avg_grade': Decimal('6.1154')},
 {'name': 'Твитоинжениринг', 'avg_grade': Decimal('5.2308')},
 {'name': 'Скаминг для мамонтов', 'avg_grade': Decimal('6.3462')},
 {'name': 'Теория относительности для чайников',
  'avg_grade': Decimal('5.5769')},
 {'name': 'Брейкдаун', 'avg_grade': Decimal('5.3846')},
 {'name': 'Пранк-психология', 'avg_grade': Decimal('6.0769')},
 {'name': 'Эго-экономика', 'avg_grade': Decimal('5.7308')}]

Отсортируем преподавателей по средней оценке по предметам, которые они ведут (если у предмета два преподавателя, мы считаем что оценки по этим предметам идут к обоим преподавателям)

In [11]:
query = '''SELECT CONCAT(last_name, ' ', first_name, ' ', patronymic) as full_name, avg_grade 
FROM
(
    SELECT AVG(grade) as avg_grade, id_teacher FROM school_database.grades 
    JOIN school_database.teachers2subjects 
    ON teachers2subjects.id_subject = grades.id_subject
    GROUP BY id_teacher
    ) avg_teach_grades

JOIN school_database.teachers
ON teachers.id_teacher = avg_teach_grades.id_teacher
ORDER BY avg_grade DESC'''

cur.execute(query)
teachers_desc = cur.fetchall()

In [12]:
teachers_desc

[{'full_name': 'Туркина Лилия Евгеньевна', 'avg_grade': Decimal('6.3462')},
 {'full_name': 'Мемовский Игнатий Игоревич', 'avg_grade': Decimal('6.1154')},
 {'full_name': 'Мамонтов Ренат Анатольевич', 'avg_grade': Decimal('6.0769')},
 {'full_name': 'Алио Елена Ивановна', 'avg_grade': Decimal('5.6731')},
 {'full_name': 'Нечитайло Сергей Владимирович',
  'avg_grade': Decimal('5.5769')},
 {'full_name': 'Нулевых Рейчел Михайловна', 'avg_grade': Decimal('5.3846')},
 {'full_name': 'Глик Евгений Юрьевич', 'avg_grade': Decimal('5.2885')},
 {'full_name': 'Доро Гупеда-Гогу -', 'avg_grade': Decimal('5.2308')},
 {'full_name': 'Шмыга Анатолий Иванович', 'avg_grade': Decimal('4.8462')}]

Найдем среди учеников Пономарёва-младшего и посмотрим, какую оценку он получил за Ускорение метаболизма

In [13]:
query = '''
    SELECT CONCAT(last_name, ' ', first_name) full_name, age, grade, name 
    FROM students, grades, subjects
    WHERE students.last_name = 'Пономарёв'
    AND subjects.name = 'Ускорение метаболизма'
    AND students.id_student = grades.id_student
    AND subjects.id_subject = grades.id_subject
    ORDER BY age LIMIT 1;
'''

cur.execute(query)
ponomarev_grade = cur.fetchall()

In [14]:
ponomarev_grade

[{'full_name': 'Пономарёв Саша',
  'age': 14,
  'grade': 2,
  'name': 'Ускорение метаболизма'}]

Исправим эту оценку на 10

In [19]:
query = '''
UPDATE school_database.grades, school_database.students, school_database.subjects
SET grades.grade = 10
WHERE students.last_name = 'Пономарёв'
AND students.first_name = 'Саша'
AND subjects.name = 'Ускорение метаболизма'
AND students.id_student = grades.id_student
AND subjects.id_subject = grades.id_subject;
'''

cur.execute(query)
con.commit()

Проверим

In [20]:
query = '''
    SELECT CONCAT(last_name, ' ', first_name) full_name, age, grade, name 
    FROM students, grades, subjects
    WHERE students.last_name = 'Пономарёв'
    AND subjects.name = 'Ускорение метаболизма'
    AND students.id_student = grades.id_student
    AND subjects.id_subject = grades.id_subject
    ORDER BY age LIMIT 1;
'''

cur.execute(query)
ponomarev_grade = cur.fetchall()
ponomarev_grade

[{'full_name': 'Пономарёв Саша',
  'age': 14,
  'grade': 10,
  'name': 'Ускорение метаболизма'}]

Удалим из базы данные для всех учеников со средней оценкой выше 5 

In [22]:
query = '''SELECT count(*) students_num FROM school_database.grades;'''

cur.execute(query)
num_students = cur.fetchall()
num_students

[{'students_num': 234}]

In [23]:
query = '''DELETE FROM school_database.grades
   WHERE grades.id_student IN
   (SELECT id_student FROM
        (SELECT id_student, AVG(grade) avg_grade
        FROM school_database.grades
        GROUP BY grades.id_student) avg_stud_grades
    WHERE avg_stud_grades.avg_grade > 5);'''

cur.execute(query)
con.commit()

In [24]:
query = '''SELECT count(*) students_num FROM school_database.grades;'''

cur.execute(query)
num_students = cur.fetchall()
num_students

[{'students_num': 81}]

Проверим

In [25]:
query = '''SELECT id_student, AVG(grade) avg_grade
FROM school_database.grades
GROUP BY grades.id_student'''

cur.execute(query)
bad_students = cur.fetchall()
bad_students

[{'id_student': 4, 'avg_grade': Decimal('4.6667')},
 {'id_student': 5, 'avg_grade': Decimal('4.8889')},
 {'id_student': 11, 'avg_grade': Decimal('4.1111')},
 {'id_student': 13, 'avg_grade': Decimal('3.7778')},
 {'id_student': 15, 'avg_grade': Decimal('5.0000')},
 {'id_student': 16, 'avg_grade': Decimal('4.5556')},
 {'id_student': 18, 'avg_grade': Decimal('4.4444')},
 {'id_student': 23, 'avg_grade': Decimal('5.0000')},
 {'id_student': 26, 'avg_grade': Decimal('4.6667')}]